In [1]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\Tsodyks Markram\\system.jl")
include("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\Tsodyks Markram\\visual_regimes.jl");

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`

In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems
E, x, y  = 0..40, 0..1,  0..1
box = E × x × y
using CairoMakie, GLMakie;

In [3]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305; const U0 = 0.265;

In [9]:
time = 500; tt = 1000; tstep = 0.001; times = [time, tt]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep)

time_int_3d = [100000, 200000]
eps_vec = [1e-10, 0.0, 0.0];

In [10]:
I0 = -1.706
p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0_1 = [8.638526524981895, 0.7320692774159767, 0.40718205935401675]
u0_2 =[7.971143689707221, 0.7053540799922068, 0.42133199265006]
println("last point  pink trajectory:$u0_1")
println("black trajectory:$u0_2")

last point  pink trajectory:[8.638526524981895, 0.7320692774159767, 0.40718205935401675]
black trajectory:[7.971143689707221, 0.7053540799922068, 0.42133199265006]


In [11]:
output = research_regime_2_traj(TM, u0_1, u0_2, p, times, integ_set,
        t_int_3d = time_int_3d,
        plot = false, onlylastpoint = false, dis_hc = true)

9-element Vector{Any}:
  3-dimensional CoupledODEs
  3-dimensional CoupledODEs
  3-dimensional StateSpaceSet{Float64} with 500001 points
  3-dimensional StateSpaceSet{Float64} with 500001 points
 5.896492168791012e-5
 5.527175753679823e-5
  Figure()
  3-dimensional StateSpaceSet{Float64} with 1 points
  Vector{ComplexF64}[[-6.382803808040045 + 0.0im, 2.173275130586992 - 13.770982609518827im, 2.173275130586992 + 13.770982609518827im]]

In [15]:
output[4][end]

3-element SVector{3, Float64} with indices SOneTo(3):
 8.827252792802732
 0.648014337530584
 0.561078276250704

In [16]:
output[8][1]

3-element SVector{3, Float64} with indices SOneTo(3):
 8.337037568242414
 0.7385031015931967
 0.4383397756224628

In [12]:
prev_1 = u0_1
prev_2 = u0_2
println("I0: $I0")
println("previous last point  pink trajectory:$prev_1")
println("previous black trajectory:$prev_2")

I0: -1.706
previous last point  pink trajectory:[8.638526524981895, 0.7320692774159767, 0.40718205935401675]
previous black trajectory:[7.971143689707221, 0.7053540799922068, 0.42133199265006]


In [13]:
data_1 = output[3]
data_2 = output[4];

In [14]:
# 3d interactive
tstart, tfinish = 200000, 500000

f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)


scale!(ax.scene, 50, 50, 1)

lines!(data_2[time_int_3d[1]:time_int_3d[2], 3], data_2[time_int_3d[1]:time_int_3d[2], 2],
        data_2[time_int_3d[1]:time_int_3d[2], 1], linewidth = 1.0, color = :black)
        
lines!(data_1[tstart:tfinish, 3], data_1[tstart:tfinish, 2],
        data_1[tstart:tfinish, 1], linewidth = 1.0, color = :deeppink)
        
scatter!(output[8][1][3], output[8][1][2], output[8][1][1], markersize = 5.0, color = :lime)

display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [62]:
ts, tf = 20000, 30000
f = Figure(resolution = (1200, 350))
axis = Axis(f[1, 1])
lines!(output[4][ts:tf, 1])
display(f)

GLMakie.Screen(...)

In [63]:
pink = lyapunovspectrum(output[1], time; Ttr = tt)
black = lyapunovspectrum(output[2], time; Ttr = tt)
println("LS  pink trajectory:$pink")
println("LS  black trajectory:$black")

LS  pink trajectory:[0.0010196312373232394, -0.69405991640678, -4.574772540613603]
LS  black trajectory:[0.0042234155238560665, -0.6968315067963479, -4.575124767499665]


In [64]:
"""# 2d projection
f = Figure(resolution = (700, 700))
ax = Axis(f[1, 1], xlabel = "x", ylabel = "y", xlabelsize = 25, ylabelsize = 25)

idx, idy = 2, 3
lines!(data_1[tstart:tfinish, idx], data_1[tstart:tfinish, idy], linewidth = 1.0, color = :deeppink)
        
scatter!(output[8][1][idx], output[8][1][idy], markersize = 5.0, color = :lime)

#scatter!(u0_1[idx], u0_1[idy], markersize = 5.0, color = :green)
display(GLMakie.Screen(), f)

f = Figure(resolution = (700, 700))
ax = Axis(f[1, 1], xlabel = "E", ylabel = "x", xlabelsize = 25, ylabelsize = 25)

idx, idy = 1, 2
lines!(data_1[tstart:tfinish, idx], data_1[tstart:tfinish, idy], linewidth = 1.0, color = :deeppink)
        
scatter!(output[8][1][idx], output[8][1][idy], markersize = 5.0, color = :lime)

#scatter!(u0_1[idx], u0_1[idy], markersize = 5.0, color = :green)
display(GLMakie.Screen(), f)

f = Figure(resolution = (700, 700))
ax = Axis(f[1, 1], xlabel = "E", ylabel = "y", xlabelsize = 25, ylabelsize = 25)

idx, idy = 1, 3
lines!(data_1[tstart:tfinish, idx], data_1[tstart:tfinish, idy], linewidth = 1.0, color = :deeppink)
        
scatter!(output[8][1][idx], output[8][1][idy], markersize = 5.0, color = :lime)

#scatter!(u0_1[idx], u0_1[idy], markersize = 5.0, color = :green)
display(GLMakie.Screen(), f)"""

"# 2d projection\nf = Figure(resolution = (700, 700))\nax = Axis(f[1, 1], xlabel = \"x\", ylabel = \"y\", xlabelsize = 25, ylabelsize = 25)\n\nidx, idy = 2, 3\nlines!(data_1[tstart:tfinish, idx], data_1[tstart:tfinish, idy], linewidth = 1.0, color = :deeppink)\n        \nscatter!(ou" ⋯ 736 bytes ⋯ "start:tfinish, idx], data_1[tstart:tfinish, idy], linewidth = 1.0, color = :deeppink)\n        \nscatter!(output[8][1][idx], output[8][1][idy], markersize = 5.0, color = :lime)\n\n#scatter!(u0_1[idx], u0_1[idy], markersize = 5.0, color = :green)\ndisplay(GLMakie.Screen(), f)"